# Machine Learning and Data Mining: Graded Lab 01
---------------

#### Grading
For this graded lab you can get a total of 15 points. These 15 points count 10% of your final grade for the course.

#### Deadline
You have 20 days to finish the lab. You must submit the graded lab the evening before the lab-session. 

- **If you have lab on Friday: Hand-in on 11.11.21, 11.59 p.m.**
- **If you have lab on Saturday: Hand-in on 12.11.21, 11.59 p.m.**

#### Submission
You have to solve the labs in groups of **two** students. If the class has an uneven number of students, there can be one group of three students. For this group, however, task 3 is graded more strictly and extra effort is expected.

Submit your GradedLab01.pynb file renamed to **LastnameStudent01_LastnameStudent02.ipynb** in moodle.   
Please submit a runnable python notebook file **and** an exported HTML with all cells executed (the results must be visible in the HTML).
All other submissions will be rejected and graded with 0 points.

#### Presentation Exercise 3
The students are completely free to decide how to solve exercise 3. If the exercise is not solved in this notebook, a detailed description with screenshots must be included. In any case, task 3 must be presented to the lecturer during class. Thereby, a live demonstration is expected and not a presentation with PowerPoint. This demonstration should take about 5 minutes.

In [ ]:
#Wichtiges:
#-lauffähiges Python notebook und mit exportierter HTMLversion -> Sonst Note1
#-Projektdatei muss so heissen: PioLogin01_Arslan02.ipynb
#-Aufgabe3 muss präsentiert werden, ca. 5min live oder über MS-Team. Kein Powerpoint. 

---

## Exercise 1: Implement a content-based recommender for the data in the movies.csv file [4 points].    
The movies.csv file contains a dataset extracted from IMDB with several attributes. For example, the title, plot, and the language of the movies.

##### __(a) The user Calvin hasn't watched any movies yet. Print the top 5 movies based on the rating attribute, as recommendations for him [0.5 points].__ 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
df = pd.read_csv('movies.csv')
pd.set_option('max_colwidth', None)
df

In [ ]:
dfSortedByRating = df.sort_values(by=['rating'], ascending=False)
dfTop = dfSortedByRating.head(5)
dfTop

##### __(b) The user Calvin hasn't still watched any movies yet. We  just know that he likes english speaking movies. Print the top 5 movies based on the rating attribute, which have only the language "English" in the languages attribute, as recommendations for him [0.5 points].__ 

In [ ]:
dfFiltered = df[df['languages'] == '[\'English\']']
dfFiltered

In [ ]:
dfSorted = dfFiltered.sort_values(by=['rating'], ascending=False)
dfTopEnglish = dfSorted.head(5)
dfTopEnglish

##### __(c) Still Calvin hasn't still watched any movies yet. However, we noticed that he took more time on the recommendations with the genres "Drama" and "Crime", therefore we assume that he is interested in this genre combination. Print the top 3 movies based on the rating attribute, which have at least the genres "Drama" and "Crime" as recommendations for him [0.5 points].__ 

In [ ]:
dfSortedCrimeDrama = df[(df['languages'] == '[\'English\']') & (df['genres'].str.contains('Drama' and 'Crime'))]
dfSortedCrimeDrama

In [ ]:
dfSortedCD = dfSortedCrimeDrama.sort_values(by=['rating'], ascending=False)
dfTopCD = dfSortedCD.head(3)
dfTopCD

##### __(d) Now, Calvin has watched the movie with the ID 110912. Print the top 3 movies based on the rating attribute, which have the same director as the watched movie with the ID 110912 as recommendations for him. The already watched movie should not be within the recommendations [0.5 points].__ 

In [ ]:
GetMovie = df[df['id'] == 110912]
GetMovie
## What are the rating attributes? just the rating or also the languages and genres? 

In [ ]:
getSortedDirectorGenre = df[df['director'] == 'Quentin Tarantino']
getSortedDirectorGenre

In [ ]:
dfWithoutPF = getSortedDirectorGenre.drop(29) #wollen nicht den Film den wir schon gesehen haben, darum entfernen
dfWithoutPF

In [ ]:
dfFinal = dfWithoutPF.sort_values(by=['rating'], ascending=False)
dfFinal

In [ ]:
dfTop3 = dfFinal.head(3)
dfTop3

##### __(d) Now, Calvin has watched the movie with the ID 1502397 and we want to recommend movies with a similar plot. Remove all stopwords from the plot attribute and split the content of the plot attribute into terms. Then calculate the term similarity between the plot of the watched movie and all others. Use data pre-processing methods wherever needed. Print the top 3 movies based on the calculated term similarity and the rating of the movie.  [2 points].__ 

In [ ]:
Movie = df[df['id'] == 1502397]
Movie

In [ ]:
import nltk
import re #Library um Zahlen zu entfernen
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
all_stopwords = stopwords.words('english') 
stopWord_list = [',', '(', ')', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '.']
all_stopwords.extend(stopWord_list) #Stopwörter werden erweitert mit zusätzlichen Zeichen


def getPlotWithoutStopwords(index):
    selectedMovie = df[df['id'] == index]
    Text = selectedMovie['plot'].to_string()
    textWithoutNum = re.sub(r'\d+', '', Text)
    text_tokens = word_tokenize(textWithoutNum) #Wörter werden tokeniziert
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords] #Iteration um alle Stopwörter zu entfernen
    return tokens_without_sw

referenceTokens = getPlotWithoutStopwords(1502397)
print(referenceTokens)

In [ ]:
distlist = []
for idx in df.index:
    tokens = getPlotWithoutStopwords(int(df.loc[idx, 'id']))
    distlist.append(nltk.edit_distance(''.join(tokens), ''.join(referenceTokens)))
df["LevenDistance"] = distlist
df

In [ ]:
dfSorted = df.sort_values(by=['LevenDistance'], ascending=False)
dfFinal = dfSorted.tail(3)
dfFinal

---

## Exercise 2: Implement Collaborative Filtering Recommenders for the data in the ratings.csv file [6 points].
##### __The ratings.csv file contains a dataset extracted from IMDB with ratings for movies from several users. Implement a user-based and an item-based collaborative filtering recommender. Use data pre-processing methods wherever needed. Print the top 3 recommended movies for each user in the dataset for the user-based and the item-based version. Don't recommend any movies the user has already watched.__ 

Setup Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
df = pd.read_csv('ratings.csv')
dm = pd.read_csv('movies.csv')
pd.set_option('max_colwidth', None)
df



    We first tried to generate the matrix of simulatiries manually but our code was really inefficient
    calculation the entire matrix took way too long so we decided to generate the user_similarity matrix
    using a pivot table and the sklearn library

    https://stackoverflow.com/questions/45387476/cosine-similarity-between-each-row-in-a-dataframe-in-python
    
    print(generateManual())


In [ ]:
#users = ["not implemented"]
#
#
#def getUsers():
#    users = df["user"].drop_duplicates().tolist()
#    return users
#
#
#def getUserReview(user):
#    # gahn dure und hole alli ratings wo user = user
#    reviews = []
#    for idx in df.index:
#        if df.loc[idx, "user"] == user:
#            reviews.append(df.loc[idx])
#
#    return pd.DataFrame(reviews, columns=["id", "user", "rating"])
#
#
#def getMovieReview(movieID):
#    # gahn dure und hole alli ratings wo Movie = Movie
#    reviews = []
#    for idx in df.index:
#        if df.loc[idx, "id"] == movieID:
#            reviews.append(df.loc[idx])
#    return pd.DataFrame(reviews, columns=["id", "user", "rating"])
#
#
#def findAB(userA, userB):
#    sameReviewedMovies = []
#    reviewlistA = getUserReview(userA)
#    reviewlistB = getUserReview(userB)
#    for idxOfA in reviewlistA.index:
#        for idxOfB in reviewlistB.index:
#            if reviewlistA.loc[idxOfA, "id"] == reviewlistB.loc[idxOfB, "id"]:
#                sameReviewedMovies.append(reviewlistA.loc[idxOfA, "id"])
#    return sameReviewedMovies
#
#
#def similarity(userA, userB):
#    same = findAB(userA, userB)
#    ratingA = []
#    # collet ratings of A
#    for id in same:
#        rating = df[(df["id"] == id) & (df["user"] == userA)]
#        ratingA.append(rating["rating"].tolist()[0])
#
#    ratingB = []
#    # collet ratings of A
#    for id in same:
#        rating = df[(df["id"] == id) & (df["user"] == userB)]
#        ratingB.append(rating["rating"].tolist()[0])
#
#    upperTerm = 0
#    for i in range(len(ratingA)):
#        upperTerm += ratingA[i] * ratingB[i]
#
#    lowerTerm1 = 0
#    allRatingsA = getUserReview(userA)["rating"].tolist()
#    for i in range(len(allRatingsA)):
#        lowerTerm1 += allRatingsA[i] ** 2
#
#    lowerTerm2 = 0
#    allRatingsB = getUserReview(userB)["rating"].tolist()
#    for i in range(len(allRatingsB)):
#        lowerTerm2 += allRatingsB[i] ** 2
#
#    lowerTerm = np.sqrt(lowerTerm1 * lowerTerm2)
#
#    similarity = upperTerm / lowerTerm
#
#    return similarity
#
#
#def applySimilarity(row):
#    for idx in row.index:
#        row.loc[idx] = similarity(row.name, idx)
#    return row
#
#
#def generateManual():
#    ratings = df.pivot_table(index="user", columns="id", values="rating", fill_value=0)
#    user_similarity = pd.DataFrame(1, columns=ratings.index.values, index=ratings.index)
#    user_similarity.apply(applySimilarity, axis=1)
#    return user_similarity
#

# Real Exercise Code

## User Based Recommendations

In [ ]:
users = df["user"].drop_duplicates().tolist()

ratings = df.pivot_table(
    index="user", columns="id", values="rating", fill_value=0
)  # Tabelle generieren mit User und Movie ID und deren Ratings

"""
In diesem Abschnitt werden die Ratings berechnet, dafür haben wir die vordefinierte Funktion cosine_similarity() verwendet von
from sklearn.metrics.pairwise import cosine_similarity
Damit man cosinus_similarity verwenden kann, braucht man eine Matrix mit allen Ratings, die wir der Methode mitgeben.

"""

userSimilarityMatrix = pd.DataFrame(
    cosine_similarity(ratings), columns=ratings.index.values, index=ratings.index
)

movies = dm.set_index("id")  # Movies indexieren

# Get all ratings of movies that the "user" did not saw
def getUnseenRatings(ratings, user):
    seen_movies = ratings.loc[user].replace(0, np.nan).dropna().index.values
    all_movies = ratings.columns.values
    unseen_movies = np.setdiff1d(all_movies, seen_movies)
    return ratings.loc[getSimilarUsers(user).index, unseen_movies]


# Multiply rating of unseen movies with simularity score of simlar users according to slide 23 of MLDM_05_RecommenderSystems_mony.pdf
# Then devide the sum by
def getRecommendationsOfRatings(ratings, user):
    unseenRatings = getUnseenRatings(ratings, user)
    sumOfRating = unseenRatings.mul(getSimilarUsers(user), axis=0).sum()
    sumOfUserSimilarity = getSimilarUsers(user).sum()
    return sumOfRating / sumOfUserSimilarity


# Sort all users expcet the "user" according the the similary to the "user"
def getSimilarUsers(user):
    return userSimilarityMatrix.loc[user].sort_values(ascending=False).drop(user)


def getUserBasedRecommendationsFor(user):
    # Get all recommendations
    recommendations = getRecommendationsOfRatings(ratings, user)
    # Get the top 3 movies
    recommendations = recommendations.sort_values(ascending=False).head(3)

    return recommendations


for user in users:
    print(
        "The user-based recommendations for "
        + user
        + " are: "
        + str(getUserBasedRecommendationsFor(user))
    )


## Item Based Recommendation

In [ ]:
users = df["user"].drop_duplicates().tolist()

ratings = df.pivot_table(
    index="user", columns="id", values="rating", fill_value=0
)  # Tabelle generieren mit User und Movie ID und deren Ratings

"""
In diesem Abschnitt werden die Ratings berechnet, dafür haben wir die vordefinierte Funktion cosine_similarity() verwendet von
from sklearn.metrics.paarweise import cosine_similarity
Damit man cosinus_similarity verwenden kann, braucht man eine Matrix mit allen Ratings, die wir der Methode mitgeben. Da wir jetz
die Movies vergleichen auf ähnlichkeit, Transponieren wir die Matrix Rating, um die Spalten auszurechnen

"""

movieSimilarityMatrix = pd.DataFrame(
    cosine_similarity(ratings.T), columns=ratings.T.index.values, index=ratings.T.index
)

movies = dm.set_index("id")  # Movies indexieren
# Get similar movies

def getSimilarMovies(ratings, user):
    seenMovies = ratings.loc[user].replace(0, np.nan).dropna().index.values
    allMovies = ratings.columns.values
    unseenMovies = np.setdiff1d(allMovies, seenMovies)
    similarMovies = movieSimilarityMatrix[allMovies].drop(seenMovies, axis=0).drop(unseenMovies, axis=1)#vergleich mit der Folie wollen nur Filme empfehlen die der User nicht hat
    return similarMovies


# Multiply rating of unseen movies with simularity score of simlar users according to slide 27 of MLDM_05_RecommenderSystems_mony.pdf
# Then devide the sum by
def getRecommendationsOfRatings(ratings, user):
    seenMovies = ratings.loc[user].replace(0, np.nan).dropna().index.values
    similarMovies = getSimilarMovies(ratings, user)
    sumOfRating = similarMovies.mul(ratings.T.loc[seenMovies, user]).sum(axis=1)
    sumOfItemSimilarity = similarMovies.sum(axis=1)
    return sumOfRating / sumOfItemSimilarity

def getItemBasedRecommendationsFor(user):
    # Get all recommendations
    recommendations = getRecommendationsOfRatings(ratings, user)
    # Get the top 3 movies
    recommendations = recommendations.sort_values(ascending=False).head(3)

    return recommendations


for user in users:
    print(
        "The Item-based recommendations for "
        + user
        + " are: "
        + str(getItemBasedRecommendationsFor(user))
    )

## Item Base Recommendation
### ZHAW SW5 Item Based Recommendation with fruits

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
df = pd.read_csv('ratingsFolie.csv')
dm = pd.read_csv('movies.csv')
pd.set_option('max_colwidth', None)
df

users = df["user"].drop_duplicates().tolist()

ratings = df.pivot_table(
    index="user", columns="id", values="rating", fill_value=0
) 


movieSimilarityMatrix = pd.DataFrame(
    cosine_similarity(ratings.T), columns=ratings.T.index.values, index=ratings.T.index
)

movies = dm.set_index("id")  

def getSimilarMovies(ratings, user):
    seenMovies = ratings.loc[user].replace(0, np.nan).dropna().index.values
    allMovies = ratings.columns.values
    unseenMovies = np.setdiff1d(allMovies, seenMovies)
    similarMovies = movieSimilarityMatrix[allMovies].drop(seenMovies, axis=0).drop(unseenMovies, axis=1)
    return similarMovies


# Multiply rating of unseen movies with simularity score of simlar users according to slide 23 of MLDM_05_RecommenderSystems_mony.pdf
# Then devide the sum by
def getRecommendationsOfRatings(ratings, user):
    seenMovies = ratings.loc[user].replace(0, np.nan).dropna().index.values
    similarMovies = getSimilarMovies(ratings, user)
    sumOfRating = similarMovies.mul(ratings.T.loc[seenMovies, user]).sum(axis=1)
    sumOfItemSimilarity = similarMovies.sum(axis=1)
    return sumOfRating / sumOfItemSimilarity

def getItemBasedRecommendationsFor(user):
    # Get all recommendations
    recommendations = getRecommendationsOfRatings(ratings, user)
    # Get the top 3 movies
    recommendations = recommendations.sort_values(ascending=False).head(3)

    return recommendations


for user in users:
    print(
        "The Item-based recommendations for "
        + user
        + " are: "
        + str(getItemBasedRecommendationsFor(user))
    )

---

## Exercise 3: Extend the Collaborative Filtering Recommender [5 points].

You are free to choose which additional task you want to do. Ideas could be:
- Implement your own UI for the Recommendation System.
- Create a blogpost on the internet describing your approach and the system (Note: Do not just upload the notebook but create a text with references to the literature and appropriate plots).
- Test alternative approaches to create recommendations and compare them.
- **... your own idea ...**

If you solve this task in this notebook, just describe your idea in a few sentences. If you do not solve it in the notebook, please provide a more detailed description and screenshots if appropriate.

This exercise must be presented during the lab session. The task is graded based on innovation, effort, result, ...

## Describe your idea here:

Our Idea is to make an analysis of our first attempt to solve the user based recommendations. Initially we tried to calculate the similarity matrix manually (without using prebuilt function from sklearn or pandas). When we tried to test our function we noticed that it took forever. We could not get any useful result. To find out whats wrong with our approach we are calculation the time complexity of our code
Lets start by determining the time complexity for all our functions:

n = number of reviews

df = pd.read_csv('ratings.csv')
dm = pd.read_csv('movies.csv')

#### Time Complexity of pandas dataframe queries
https://stackoverflow.com/questions/45240803/pandas-dataframe-search-is-linear-time-or-constant-time


***
### getUserReview()

#### Description
This function returns all the reviews that a specific user created.
This function is used later to calculate the cosine similarity

#### Complexity

1 for loop iterating over all users = O(n)

Total : = (n)


In [ ]:
def getUserReview(user):
    reviews = []
    for idx in df.index:
        if df.loc[idx, "user"] == user:
            reviews.append(df.loc[idx])

    return pd.DataFrame(reviews, columns=["id", "user", "rating"])

***
### findAB
#### Description
This function searches for movies that both userA and userB have reviewed.
We need this function to calculate the upper fraction term of the cosine similarity formla
#### Complexity

\+ getUserReview = O(n)

\+ getUserReview = O(n)

\+ 2 nested for loops over reviews = O(n^2)

= 

O(n) + O(n) + O(n^2)

Total :  O(n^2)


In [ ]:
def findAB(userA, userB):
    sameReviewedMovies = []
    reviewlistA = getUserReview(userA)
    reviewlistB = getUserReview(userB)
    for idxOfA in reviewlistA.index:
        for idxOfB in reviewlistB.index:
            if reviewlistA.loc[idxOfA, "id"] == reviewlistB.loc[idxOfB, "id"]:
                sameReviewedMovies.append(reviewlistA.loc[idxOfA, "id"])
    return sameReviewedMovies


***
### similarity
#### Description
This function calculates the cosine similarity between two users according to the formula on the lectue slides. Here we make use of the functions defined above.
#### Complexity
findAB = O(n^2)

\+ for loop over ratings + query over dataframe = O(n^2)

\+ for loop over ratings + query over dataframe = O(n^2)

\+ for loop over ratings =  O(n)

\+ getUserReview = O(n)

\+ for loop over ratings =  O(n)

\+ getUserReview = O(n)

\+ for loop over ratings =  O(n)

= O(n^2) + O(n^2) + O(n) + O(n) + O(n) + O(n)

= 2* O(n^2) + 5*O(n)

Total : O(n^2)

In [ ]:
def similarity(userA, userB):
    same = findAB(userA, userB)
    ratingA = []
    # collet ratings of A
    for id in same:
        rating = df[(df["id"] == id) & (df["user"] == userA)]
        ratingA.append(rating["rating"].tolist()[0])

    ratingB = []
    # collet ratings of A
    for id in same:
        rating = df[(df["id"] == id) & (df["user"] == userB)]
        ratingB.append(rating["rating"].tolist()[0])

    upperTerm = 0
    for i in range(len(ratingA)):
        upperTerm += ratingA[i] * ratingB[i]

    lowerTerm1 = 0
    allRatingsA = getUserReview(userA)["rating"].tolist()
    for i in range(len(allRatingsA)):
        lowerTerm1 += allRatingsA[i] ** 2

    lowerTerm2 = 0
    allRatingsB = getUserReview(userB)["rating"].tolist()
    for i in range(len(allRatingsB)):
        lowerTerm2 += allRatingsB[i] ** 2

    lowerTerm = np.sqrt(lowerTerm1 * lowerTerm2)

    similarity = upperTerm / lowerTerm

    return similarity


***
### applySimilarity
#### Description
This function is just a small helper function to apply the cosine similarity to a row in a matrix. 
This is used in the next step
#### Complexity
\+ similarity = O(n^2)

\+ for over cols in a row = num_of_cols * O(n^2)

= (detailed calculation in next chapter)

In [ ]:
def applySimilarity(row):
    for idx in row.index:
        row.loc[idx] = similarity(row.name, idx)
    return row

***
### generateSimularityMatrix
#### Description
This function generates a pivot matrix of the users and then calculates the cosine similarity between them.
As a first try we did this without any optimization for each collumn and row

#### Complexity
\+ generating pivot table with m^2 entries

\+call applySimilarity for m rows where each row has m cols which results in :

\+O(n^2) * applySimilarity 

= O(n^2) * O(n^2) 

Total : O(n^4)


In [ ]:
def generateSimularityMatrix():
    ratings = df.pivot_table(index="user", columns="id", values="rating", fill_value=0)
    user_similarity = pd.DataFrame(1, columns=ratings.index.values, index=ratings.index)
    user_similarity.apply(applySimilarity, axis=1)
    return user_similarity

## Conclusion
Assuming one simple claculation including all the array access takes about 1msec. (This is just a guess, we did some testing : calculatin 1000 simple cosines takes about 1 msec).

We got about about 80 unique users in our data. This means for our rough guess our function takes (80\*80\*80\*80)\*1msec = 41 000 000 msec = 12h
Even though this is the maximum possible execution time we could not get any resulat with our first attemped. Thats why we dropped this attempt and used prebuilt function.

To conclude this enitre thing: next time just use the prebuilt function for stuff like this since these are proballby heavily optimized already. Using the prebuilt functions the similarity matrix is built in an instant





